In [ ]:
### Importing Libraries

import glob
import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#folder location for all the files
file_loc = '/content/drive/MyDrive/Information Retrieval/A1/data/*' 

In [ ]:
# all the stopwords from nltk corpus
stps = stopwords.words('english')

In [ ]:
# building a file list 
file_list = []
for file in glob.glob(file_loc):
  file_list.append(file)
print(len(file_list))

1133


In [ ]:
file_list[56]

'/content/drive/MyDrive/Information Retrieval/A1/data/lifeimag.hum'

In [ ]:
#Prepprocessing a given string
def preprocess(text):
  lemmatizer = WordNetLemmatizer() 
  # converting into lower case
  text = text.lower()
  #removing hyperlinks
  text = re.sub(r"http\S+", " ", text)
  #removing punctuations
  text = re.sub(r'[^\w\s]', ' ', text)
  #tokenizing
  text_tokens = nltk.word_tokenize(text)
  new_tokens = []
  #lemmatizing
  text_tokens = [lemmatizer.lemmatize(word) for word in text_tokens]
  #removing stopwords
  for token in text_tokens:
        if token not in stps:
            new_tokens.append(token)

  return new_tokens


In [ ]:
#Building inverted index
file_cont = [] 

inverted_index = {}   # a dictionary that represents the inverted index
i = 0
for file_r in file_list:
  #opening file in reading mode
  file_o = open(file_r , "r", encoding='latin-1')
  # extracting contents of the file into text
  text = file_o.read()
  #adding words to the inverted index after preprocessing
  for word in set(preprocess(text)):
    if word in inverted_index:
      inverted_index[word].append(i)
    else:
        inverted_index[word] = [i]
  i+=1


In [ ]:
len(inverted_index.keys())

65099

In [ ]:
# OR Operation
def orOp(list1, list2):
  len1 = len(list1)
  len2 = len(list2)

  res = []
  n_comp = 0
# Base Cases and comparisons
  if len1 == 0 and len2 == 0:
    n_comp += 2
    return (res,n_comp)
  n_comp += 2
  if len1 == 0 and len2 != 0:
    n_comp += 2
    return (list2,n_comp)
  n_comp += 2
  if len1 != 0 and len2 == 0:
    n_comp += 2
    return (list1,n_comp)
  n_comp += 2
  # adding a +inf element
  list1.append(1200)
  list2.append(1200)

  i = 0
  j = 0
  #O(n) time union operation for sorted lists
  while(i <= len1 and j <= len2):

    if(list1[i] == list2[j]):
      res.append(list1[i])
      i += 1
      j += 1
      n_comp += 1
    
    elif list1[i] < list2[j]:
      res.append(list1[i])
      i += 1
      n_comp += 2

    else:
      res.append(list2[j])
      j += 1
      n_comp += 2
  
  res = [i for i in res if i != 1200]
 
  return (res,n_comp)
  



In [ ]:
# AND Operation
def andOp(list1, list2):
  len1 = len(list1)
  len2 = len(list2)

  res = []
  n_comp = 0
#Base Cases
  if len1 == 0 and len2 == 0:
    n_comp += 2
    return (res,n_comp)
  n_comp += 2
  if len1 == 0 and len2 != 0:
    n_comp += 2
    return (res,n_comp)
  n_comp += 2
  if len1 != 0 and len2 == 0:
    n_comp += 2
    return (res,n_comp)
  n_comp += 2
# adding a +inf element  
  list1.append(1200)
  list2.append(1200)

  i = 0
  j = 0
#O(n) time intersection operation for sorted lists
  while(i <= len1 and j <= len2):

    if(list1[i] == list2[j]):
      res.append(list1[i])
      i += 1
      j += 1
      n_comp += 1
    
    elif list1[i] < list2[j]:
      i += 1
      n_comp += 2

    else:
      j += 1
      n_comp += 2
  
  res = [i for i in res if i != 1200]
  print(res)
  return (res,n_comp)
  



In [ ]:
# Not Operation
def notOp(list1):
  res = []
  for i in range(len(file_list)):
    if i not in list1:
      res.append(i)
  return res

In [ ]:
#OR Not Operation
def orNotOp(list1, list2):
  notL2 = notOp(list2)
  L1orNotL2, orNotComp = orOp(list1,notL2)
  
  return (L1orNotL2,orNotComp)

In [ ]:
#AND NOT Operation
def andNotOp(a,b):
  # b = notOp(b)
  # list_3,y = andOp(a,b)
  list_1=a
  list_2=b
  list_3=[]
  ptr_1=0
  ptr_2=0
  comp = 0
  
  while(ptr_1<len(list_1) and ptr_2<len(list_2)):
    if(list_1[ptr_1]<list_2[ptr_2]):
      list_3.append(list_1[ptr_1])
      ptr_1+=1
      comp += 1
    elif(list_1[ptr_1]==list_2[ptr_2]):
      ptr_1+=1
      ptr_2+=1
      comp += 2
    else:
      ptr_2+=1
      comp += 2
  while(ptr_1<len(list_1)):
    list_3.append(list_1[ptr_1])
    ptr_1+=1
    

  return (list_3,comp)

In [ ]:
#Querry Running 
def runQuerry(qry,opArr):

  qryArr = preprocess(qry)
  i = 0
 
  if len(qryArr) != len(opArr)+1:
    return "can't process"
  res = inverted_index[qryArr[0]]
  t_comp = 0
  for i in range(len(opArr)):
    doc_list = inverted_index[qryArr[i+1]]
    if opArr[i] == "OR":
      res,comp = orOp(res,doc_list)
    elif opArr[i] == "AND":
      res,comp = andOp(res,doc_list)
    elif opArr[i] == "AND NOT":
      res,comp = andNotOp(res,doc_list)
    elif opArr[i] == "OR NOT":
      res,comp = orNotOp(res,doc_list)
    else:
      print("Wrong Operation!!!!")
      break
    t_comp += comp
  # displaying results
  print("Number of documents matched: ", len(res))
  print("The number of comparisons are ",t_comp)
  print("The documents matched are : ")
  print(res)
  for i in res:
    print(file_list[i]) 

  


In [ ]:
qry = ""
op = ["OR NOT","AND NOT"]
runQuerry(qry,op)

"can't process"